In [ ]:
#!pip install litellm

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
from litellm import completion
import os


openaillm = completion(
  model="openai/gpt-4o",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)

In [ ]:
print(openaillm)

In [ ]:
from litellm import acompletion
import asyncio


async def test_get_response():
    user_message = "Hello, how are you?"
    messages = [{"content": user_message, "role": "user"}]
    response = await acompletion(model="openai/gpt-4o", messages=messages)
    return response

response = asyncio.run(test_get_response())
print(response)

In [ ]:

from litellm import acompletion
import asyncio

async def test_get_response():
    user_message = "Hello, how are you?"
    messages = [{"content": user_message, "role": "user"}]
    response = await acompletion(model="openai/gpt-4o", messages=messages)
    return response

response = await test_get_response()
print(response)




In [ ]:
import nest_asyncio; nest_asyncio.apply()
response2 = asyncio.run(test_get_response())
print(response2)

### Quick summary

Top-level await: Jupyter/IPython kernels support awaiting coroutines directly in cells (preferred).
Problem: asyncio.run() raises RuntimeError if an event loop is already running (common in notebooks).
Fix: either use top-level await, or install/apply nest_asyncio to allow asyncio.run inside a running loop.
Install & use nest_asyncio

In [ ]:
# Example: top-level await in a notebook cell
from litellm import acompletion

async def test_get_response():
    msgs = [{"content": "Hello, how are you?", "role": "user"}]
    return await acompletion(model="openai/gpt-4o", messages=msgs)

# In a notebook cell that supports top-level await:
resp = await test_get_response()
print(resp)

In [ ]:
# Install once:
#!pip install nest_asyncio

# Then in a notebook cell:
import nest_asyncio, asyncio
nest_asyncio.apply()

from litellm import acompletion

async def test_get_response():
    msgs = [{"content": "Hello, how are you?", "role": "user"}]
    return await acompletion(model="openai/gpt-4o", messages=msgs)

# Now asyncio.run will not raise RuntimeError
resp = asyncio.run(test_get_response())
print(resp)

# LITE LLM PROXY

In [ ]:
#!pip install litellm[proxy]

In [ ]:
!pip install presidio_analyzer

In [ ]:
from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import SpacyNlpEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider

# Configuration for the NLP engine
configuration = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}]
}

# Create NLP engine
provider = NlpEngineProvider(nlp_configuration=configuration)
nlp_engine = provider.create_engine()

# Plug NLP engine into Analyzer
analyzer = AnalyzerEngine(nlp_engine=nlp_engine, supported_languages=["en"])

# Test input
text = "My name is Rakesh garapati  and my email is rakesh@example.com and PAN is xxxx-xxxx-xxxx 49296919 26368685"
results = analyzer.analyze(text=text, entities=[], language="en")

print("Analysis complete. ", results)

for result in results:
    print(f"{result.entity_type}: {text[result.start:result.end]} (score={result.score:.2f})")

#print("Analysis complete. ", results)

In [ ]:
# Replace ranges from end->start to avoid index shifts and wrap with tags
tagged = text
for r in sorted(results, key=lambda x: x.start, reverse=True):
    start, end = r.start, r.end
    tag = r.entity_type  # e.g. "PERSON" or "EMAIL_ADDRESS"
    tagged = tagged[:start] + f"<{tag}> </{tag}> " + tagged[end:]

print(tagged)

In [ ]:
import os, requests

ANALYZER = os.environ.get("PRESIDIO_ANALYZER_API_BASE", "http://127.0.0.1:5002")
ANONYMIZER = os.environ.get("PRESIDIO_ANONYMIZER_API_BASE", "http://127.0.0.1:5001")
text = "My name is John Doe and my email is john@example.com"

# 1) Analyze
a_resp = requests.post(f"http://127.0.0.1:5002/analyze", json={"text": text, "language": "en"}, timeout=30)
a_resp.raise_for_status()
results = a_resp.json()  # list of detection results

# Convert detections to anonymizer entities and build anonymizers_config (mask each detected type)
entities = [{"start": r["start"], "end": r["end"], "entity_type": r.get("entity_type", r.get("type"))} for r in results]
anonymizers_config = {e["entity_type"]: {"type": "mask", "masking_char": "*"} for e in entities}


In [ ]:
print("Anonymizers config:", anonymizers_config)

In [ ]:

# 2) Anonymize
payload = {
    "text": text,
    "entities": entities,
    "analyzer_results": results,  
    "anonymizers_config": anonymizers_config,
}
anon_resp = requests.post(f"{ANONYMIZER}/anonymize", json=payload, timeout=30)
anon_resp.raise_for_status()
print(anon_resp.json().get("text") or anon_resp.json())  # masked text

In [ ]:
#Step 1: Start litellm proxy
#$ litellm --model huggingface/bigcode/starcoder
#$ litellm --model openai/gpt-4o

#INFO: Proxy running on http://0.0.0.0:4000


#https://genai.owasp.org/llm-top-10/

#"Input should be 
# 'CREDIT_CARD', 'CRYPTO', 'DATE_TIME', 'EMAIL_ADDRESS', 'IBAN_CODE', 'IP_ADDRESS', 'NRP', 
# 'LOCATION', 'PERSON', 'PHONE_NUMBER', 'MEDICAL_LICENSE', 'URL', 'US_BANK_NUMBER', 'US_DRIVER_LICENSE', 
# 'US_ITIN', 'US_PASSPORT', 'US_SSN', 'UK_NHS', 'UK_NINO', 'ES_NIF', 'ES_NIE', 'IT_FISCAL_CODE', 
# 'IT_DRIVER_LICENSE', 'IT_VAT_CODE', 'IT_PASSPORT', 'IT_IDENTITY_CARD', 'PL_PESEL', 'SG_NRIC_FIN', 
# 'SG_UEN', 'AU_ABN', 'AU_ACN', 'AU_TFN', 'AU_MEDICARE', 'IN_PAN', 'IN_AADHAAR', 
# 'IN_VEHICLE_REGISTRATION', 'IN_VOTER', 'IN_PASSPORT' or 'FI_PERSONAL_IDENTITY_CODE'",



In [ ]:
litellm --config config.yaml --detailed_debug

curl -v http://127.0.0.1:4000/health
curl -v http://127.0.0.1:4000/docs   # FastAPI docs, if enabled
curl -v http://127.0.0.1:4000/v1/models

  ```bash
    curl -X GET "http://localhost:4000/guardrails/list" -H "Authorization: Bearer <your_api_key>"
    ```


OPENAI_API_KEY="sk-REPLACE_WITH_NEW_KEY"
LANGCHAIN_API_KEY="lsv2_REPLACE_WITH_NEW_KEY"
# Set Presidio analyzer URL (map to container port on host)
PRESIDIO_ANALYZER_API_BASE="http://127.0.0.1:3000"
# optional if your deploy requires an API key
PRESIDIO_ANALYZER_API_KEY="replace_if_required"


curl.exe http://127.0.0.1:4000/health
curl.exe -H "Authorization: Bearer $env:OPENAI_API_KEY" http://127.0.0.1:4000/guardrails/list

# apply guardrail (example)
curl.exe -X POST "http://127.0.0.1:4000/guardrails/apply_guardrail" `
  -H "Content-Type: application/json" `
  -H "Authorization: Bearer $env:OPENAI_API_KEY" `
  -d '{"guardrail_name":"presidio-pii","text":"My name is John Doe and my email is john@example.com","language":"en","entities":["NAME","EMAIL"]}'



In [ ]:
import requests
import json

url = "http://localhost:4000/v1/chat/completions"

payload = json.dumps({
  "model": "gpt-3.5-turbo",
  "messages": [
    {
      "role": "user",
      "content": "hi my email is ishaan@berri.ai , what is my email? and tell me a joke"
    }
  ],
  "guardrails": [
    "presidio-pii"
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Content-Type': 'application/json'
  
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


In [ ]:
import openai
client = openai.OpenAI(
    api_key="anything",
    base_url="http://127.0.0.1:4000"
)

# request sent to model set on litellm proxy, `litellm --model`
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages = [
        {
            "role": "user",
            "content": "my name is rakesh email is rakesh@gmail.com  and phone number is 9030777879"
        }
    ],
    extra_body={ 
        "metadata": {
            "guardrails": ["presidio-pii"],
            "guardrail_config": {"language": "es"}
        }
    }
)
print(response.choices[0].message.content)

In [ ]:
import requests
import json

url = "http://127.0.0.1:4000/apply_guardrail"

payload = json.dumps({
  "guardrail_name": "presidio-pii",
  "text": "My name is John Doe and my email is john@example.com and aadhar is 6373 4567 2345",
  "language": "string",
  "entities": [
    "CREDIT_CARD"
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


In [ ]:
import openai # openai v1.0.0+
client = openai.OpenAI(base_url="http://127.0.0.1:4000") # set proxy to base_url
# request sent to model set on litellm proxy, `litellm --model`
response = client.chat.completions.create(model="", messages = [
    {
        "role": "user",
        "content": "this is a test request, write a very long  poem"
    }
],
stream=True,
)

# stream is an iterator of chunks/events
for chunk in response:
    # chunk is dict-like; content often at chunk.choices[0].delta.get("content")
    try:
        delta = chunk.choices[0].delta
        text = delta.get("content") or delta.get("role") or ""
    except Exception:
        text = getattr(chunk, "text", "") or str(chunk)
    if text:
        print(text, end="", flush=True)
print()

In [ ]:


import requests
import json

url = "http://127.0.0.1:4000/guardrails/apply_guardrail"

payload = json.dumps({
  "guardrail_name": "presidio-pii",
  "text": "My name is John Doe and my email is john@example.com and ph no is 9030767789 adhar no is 1234 5678 9123 and credit card no is 4111 1111 1111 1111",
  "language": "en",
  "entities": [
    "PERSON",
    "EMAIL_ADDRESS",
    "IN_AADHAAR",
    "CREDIT_CARD"
  ]
})
headers = {
  'Authorization': 'Bearer XXX',
  'Content-Type': 'application/json',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)
